In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 11.5MB/s 
     |████████████████████████████████| 3.3MB 52.5MB/s 
     |████████████████████████████████| 901kB 54.6MB/s 


In [ ]:
#######################################
### -------- Load libraries ------- ###
# Load Huggingface transformers
from transformers import TFBertModel, BertConfig, BertTokenizer
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Define Model Structure and Load Best Weights

In [ ]:
model_name = 'hfl/chinese-roberta-wwm-ext-large'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert = TFBertModel.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
max_length = 50

In [ ]:
roberta = bert.layers[0]

In [ ]:
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

#Load the Transformers BERT model as a layer in a Keras model
roberta_model = roberta(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(roberta_model, training=False)

#output
fc1 = Dense(units=512,activation='relu', kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='fc1')(pooled_output)
fc1_output = Dropout(0.1, name='fc1_output')(fc1)
subjectivity = Dense(units=2,activation='softmax',kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='subjectivity')(fc1_output)

output = {'label': subjectivity}

In [ ]:
#combined
model = Model(inputs=inputs, outputs=output, name='ROBERTA')
model.summary()

Model: "ROBERTA"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 50)]              0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 325522432 
_________________________________________________________________
pooled_output (Dropout)      (None, 1024)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 512)               524800    
_________________________________________________________________
fc1_output (Dropout)         (None, 512)               0         
_________________________________________________________________
subjectivity (Dense)         (None, 2)                 1026      
Total params: 326,048,258
Trainable params: 326,048,258
Non-trainable params: 0
_____________________________________________

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamWeightDecay

# define the optimizer
optimizer = AdamWeightDecay(lr = 2e-5, weight_decay_rate=0.01)

In [ ]:
loss = {'label': CategoricalCrossentropy(from_logits = True)}
metric = {'label': CategoricalAccuracy('accuracy')}

In [ ]:
model.compile(optimizer = optimizer, loss = loss, metrics = metric)

In [ ]:
model.load_weights('/content/gdrive/MyDrive/subbert.ckpt')

# Load Data, Inference and Filter

In [ ]:
files = glob.glob("gdrive/MyDrive/RMBI_FYP/August data/*.csv")
for f in files:
    data = pd.read_csv(f)
    data['ct'] = data['ct'].astype('str')
    data_x = tokenizer(text=data['ct'].to_list(), add_special_tokens=True, max_length=max_length, truncation=True, padding=True, return_tensors='tf', return_token_type_ids = False, return_attention_mask = False, verbose = True)
    y_pred = model.predict(x={'input_ids': data_x['input_ids']})
    y_pred_label = np.argmax(y_pred['label'], axis=1)
    data[['subjectivity']] = y_pred_label
    data = data[data['subjectivity'] == 1]
    data.drop(labels = ['Unnamed: 0', 'subjectivity'], axis=1, inplace=True)
    data.to_csv(f,index=False)

# Old Code For Reference

In [ ]:
data = pd.read_csv("gdrive/MyDrive/RMBI_FYP/August data/20160801.csv")

In [ ]:
data['ct'] = data['ct'].astype('str') 
data.head()

,Unnamed: 0,_id,au,ct,hl,st
0,0,201608013044537,ZDSZWDJN,一萬多億的盤子，二萬多億的公司資產\n,一萬多億的盤子，二萬多億的公司資產,600028.sh
1,2,201608013044636,江西贛州網友,"京東方三大股東承諾半年不減持在7月28號到期，這些王八蛋7月28和29號連著倆天出貨,成本2...",告全體散戶朋友書,000725.sz
2,5,201608013044693,美的克星,上央視，跌不止，坑散戶，真無恥\n,上央視，跌不止，坑散戶，真無恥,000725.sz
3,8,201608013044698,竹凡語北燦,暴跌後怎麼辦？重大盤不如重個股，數股異軍突起或成明日之星，20點收益等待入場信號，可跟進之前\n,暴跌後怎麼辦？重大盤不如重個股，數股異軍突起或成明日之星，20點收益等待入場信號,600109.sh
4,12,201608013044734,股友ssMPnT,每個股吧都有京東方打的廣告！\n,每個股吧都有京東方打的廣告！,000725.sz


# Inference

In [ ]:
data_x = tokenizer(text=data['ct'].to_list(), add_special_tokens=True, max_length=max_length, truncation=True, padding=True, return_tensors='tf', return_token_type_ids = False, return_attention_mask = False, verbose = True)

In [ ]:
y_pred = model.predict(x={'input_ids': data_x['input_ids']})

In [ ]:
y_pred

{'label': array([[0.00431316, 0.9956868 ],
        [0.00150657, 0.99849343],
        [0.00150552, 0.9984945 ],
        ...,
        [0.00337283, 0.9966272 ],
        [0.00173076, 0.9982692 ],
        [0.12868291, 0.8713171 ]], dtype=float32)}

In [ ]:
y_pred_label = np.argmax(y_pred['label'], axis=1)
y_pred_label

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
data[['subjectivity']] = y_pred_label

In [ ]:
data = data[data['subjectivity'] == 1]

In [ ]:
data.drop(labels = ['Unnamed: 0', 'subjectivity'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
data.to_csv('test.csv',index=False)